In [42]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [44]:
# Указываем пути к папкам (скорректируйте, если у вас другие названия)
train_dir = 'dataset/train'
valid_dir = 'dataset/valid'
test_dir  = 'dataset/test'   # В test лежат просто изображения



In [55]:
# Параметры
IMG_SIZE = (224, 224)   # Подходящий размер для VGG16
BATCH_SIZE = 32
NUM_EPOCHS = 2         

In [56]:
# 1) Создаём генераторы изображений
# Для обучения используем аугментацию
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [57]:
# Для валидации и теста обычно достаточно только предобработки
valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen  = ImageDataGenerator(preprocessing_function=preprocess_input)

# 2) Создаём генераторы (flow_from_directory)
train_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'  # несколько классов
)

Found 6552 images belonging to 102 classes.


In [58]:
valid_generator = valid_datagen.flow_from_directory(
    directory=valid_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 818 images belonging to 102 classes.


In [68]:
test_generator = test_datagen.flow_from_directory(
    directory='dataset',
    classes=['test'],        # Явно указываем, что надо брать файлы из папки "test"
    target_size=IMG_SIZE,
    class_mode=None,
    batch_size=1,
    shuffle=False
)


Found 819 images belonging to 1 classes.


In [69]:
# 3) Загружаем предобученную модель (VGG16) без верхних слоёв
base_model = VGG16(
    weights='imagenet',
    include_top=False, 
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)
)


In [70]:
# 4) "Замораживаем" веса базовой модели
for layer in base_model.layers:
    layer.trainable = False


In [71]:
# 5) Добавляем свои слои поверх базовой модели
x = base_model.output
x = GlobalAveragePooling2D()(x)   # вместо Flatten можно GlobalAveragePooling2D
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
# Выходной слой: количество нейронов = числу классов
num_classes = train_generator.num_classes
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)


In [72]:
# 6) Компилируем модель
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [73]:
# 7) Обучаем модель
history = model.fit(
    train_generator,
    epochs=NUM_EPOCHS,
    validation_data=valid_generator
)

Epoch 1/2
205/205 ━━━━━━━━━━━━━━━━━━━━ 77s 370ms/step - accuracy: 0.0290 - loss: 12.8773 - val_accuracy: 0.3289 - val_loss: 3.6695
Epoch 2/2
205/205 ━━━━━━━━━━━━━━━━━━━━ 146s 711ms/step - accuracy: 0.1574 - loss: 7.3922 - val_accuracy: 0.5562 - val_loss: 2.1784


In [74]:
# 8) Оцениваем модель на валидационной выборке
val_loss, val_acc = model.evaluate(valid_generator)
print(f"Validation accuracy: {val_acc:.4f}")


26/26 ━━━━━━━━━━━━━━━━━━━━ 15s 576ms/step - accuracy: 0.5615 - loss: 2.1551
Validation accuracy: 0.5562


In [76]:
# 9) Предсказываем результаты на тестовой выборке
test_generator.reset()  # на всякий случай сбрасываем указатель
preds = model.predict(test_generator, verbose=1)

2025-03-02 00:50:26.646327: W tensorflow/core/framework/op_kernel.cc:1827] INVALID_ARGUMENT: TypeError: `generator` yielded an element that did not match the expected structure. The expected structure was (tf.float32,), but the yielded element was [[[[ -66.939      -62.779      -80.68     ]
   [ -67.939      -62.779      -82.68     ]
   [ -68.939      -62.779      -84.68     ]
   ...
   [  79.061      112.221       58.32     ]
   [ 127.061      138.22101    107.32     ]
   [ 123.061      123.221      108.32     ]]

  [[ -68.939      -65.779      -83.68     ]
   [ -72.939      -68.779      -88.68     ]
   [ -76.939      -72.779      -93.68     ]
   ...
   [  99.061      134.22101     85.32     ]
   [ 117.061      138.22101    107.32     ]
   [ 115.061      126.221      107.32     ]]

  [[ -78.939      -77.779      -96.68     ]
   [ -76.939      -75.779      -94.68     ]
   [ -74.939      -72.779      -93.68     ]
   ...
   [ 109.061      138.22101    106.32     ]
   [ 105.061      138.2

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_1_device_/job:localhost/replica:0/task:0/device:CPU:0}} TypeError: `generator` yielded an element that did not match the expected structure. The expected structure was (tf.float32,), but the yielded element was [[[[ -66.939      -62.779      -80.68     ]
   [ -67.939      -62.779      -82.68     ]
   [ -68.939      -62.779      -84.68     ]
   ...
   [  79.061      112.221       58.32     ]
   [ 127.061      138.22101    107.32     ]
   [ 123.061      123.221      108.32     ]]

  [[ -68.939      -65.779      -83.68     ]
   [ -72.939      -68.779      -88.68     ]
   [ -76.939      -72.779      -93.68     ]
   ...
   [  99.061      134.22101     85.32     ]
   [ 117.061      138.22101    107.32     ]
   [ 115.061      126.221      107.32     ]]

  [[ -78.939      -77.779      -96.68     ]
   [ -76.939      -75.779      -94.68     ]
   [ -74.939      -72.779      -93.68     ]
   ...
   [ 109.061      138.2210

2025-03-02 00:50:26.663181: W tensorflow/core/framework/op_kernel.cc:1827] INVALID_ARGUMENT: TypeError: `generator` yielded an element that did not match the expected structure. The expected structure was (tf.float32,), but the yielded element was [[[[ -83.939      -57.779      -90.68     ]
   [ -83.939      -55.779      -93.68     ]
   [ -79.939      -49.779      -93.68     ]
   ...
   [-103.939     -100.779     -118.68     ]
   [-100.939      -96.779     -114.68     ]
   [-103.939      -97.779     -116.68     ]]

  [[ -79.939      -53.779      -86.68     ]
   [ -76.939      -48.779      -86.68     ]
   [ -74.939      -45.779      -87.68     ]
   ...
   [-102.939      -95.779     -114.68     ]
   [-103.939      -97.779     -116.68     ]
   [ -94.939      -79.779      -99.68     ]]

  [[ -80.939      -53.779      -88.68     ]
   [ -79.939      -51.779      -89.68     ]
   [ -78.939      -50.779      -89.68     ]
   ...
   [-103.939      -90.779     -112.68     ]
   [ -97.939      -74.7

In [78]:
def wrap_generator(gen):
    for batch in gen:
        yield (batch,)  # оборачиваем каждую порцию в кортеж

wrapped_test_generator = wrap_generator(test_generator)
preds = model.predict(wrapped_test_generator, steps=len(test_generator), verbose=1)


819/819 ━━━━━━━━━━━━━━━━━━━━ 15s 17ms/step


In [79]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

test_dir = 'dataset/test'
test_images = sorted(os.listdir(test_dir))  # список файлов
IMG_SIZE = (224, 224)

pred_classes = []
for filename in test_images:
    img_path = os.path.join(test_dir, filename)
    # Загружаем изображение и изменяем размер
    img = load_img(img_path, target_size=IMG_SIZE)
    x = img_to_array(img)
    x = preprocess_input(x)  # та же функция, что и для обучения
    x = np.expand_dims(x, axis=0)  # делаем батч размером 1

    preds = model.predict(x)
    pred_class = np.argmax(preds, axis=1)[0]
    pred_classes.append(pred_class)

print(pred_classes)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━

In [80]:
print("Классы (label -> индекс):", train_generator.class_indices)

Классы (label -> индекс): {'1': 0, '10': 1, '100': 2, '101': 3, '102': 4, '11': 5, '12': 6, '13': 7, '14': 8, '15': 9, '16': 10, '17': 11, '18': 12, '19': 13, '2': 14, '20': 15, '21': 16, '22': 17, '23': 18, '24': 19, '25': 20, '26': 21, '27': 22, '28': 23, '29': 24, '3': 25, '30': 26, '31': 27, '32': 28, '33': 29, '34': 30, '35': 31, '36': 32, '37': 33, '38': 34, '39': 35, '4': 36, '40': 37, '41': 38, '42': 39, '43': 40, '44': 41, '45': 42, '46': 43, '47': 44, '48': 45, '49': 46, '5': 47, '50': 48, '51': 49, '52': 50, '53': 51, '54': 52, '55': 53, '56': 54, '57': 55, '58': 56, '59': 57, '6': 58, '60': 59, '61': 60, '62': 61, '63': 62, '64': 63, '65': 64, '66': 65, '67': 66, '68': 67, '69': 68, '7': 69, '70': 70, '71': 71, '72': 72, '73': 73, '74': 74, '75': 75, '76': 76, '77': 77, '78': 78, '79': 79, '8': 80, '80': 81, '81': 82, '82': 83, '83': 84, '84': 85, '85': 86, '86': 87, '87': 88, '88': 89, '89': 90, '9': 91, '90': 92, '91': 93, '92': 94, '93': 95, '94': 96, '95': 97, '96': 98,

In [81]:
# 10) Сохраняем модель (при необходимости)
model.save("flower_model.h5")